In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import win32com.client as win32
import pandas as pd
import time
servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)


# importar/visualizar base de dados

tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

def verificar_tem_termos_banidos(lista_termos_banidos, nome):
    tem_termos_banidos = False
    for termo in lista_termos_banidos:
        if termo in nome:
            tem_termos_banidos = True
    return tem_termos_banidos

def verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome):
    tem_todos_termos_produto = True
    for termo in lista_termos_nome_produto:
        if termo not in nome:
            tem_todos_termos_produto = False
    return tem_todos_termos_produto

def busca_google_shopping(navegador, produto, termos_banidos, preco_minimo, preco_maximo):
    produto = produto.lower()
    lista_termos_nome_produto = produto.split(' ')
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    lista_ofertas = []

    # entrar no google
    navegador.get("https://www.google.com/")
    
    # entrar na aba shopping
    navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto, Keys.ENTER)
    elementos = navegador.find_elements('class name', 'hdtb-mitem')

    for item in elementos:
        if 'Shopping' in item.text:
            item.click()
            break
            
    # pegar as informações do produto
    lista_resultados = navegador.find_elements('class name', 'i0X6df')

    for resultado in lista_resultados:
        nome = resultado.find_element('class name', 'tAxDx').text
        nome = nome.lower()
        
        # analisar se nome não tem nenhum dos termos banidos
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)

        # analisar se nome tem todos os termos do produto
        tem_todos_termos_produto = verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome)

        # selecionar só os elementos que tem_termos_banidos = False e ao mesmo tempo tem_todos_termos_produto = True
        if not tem_termos_banidos and tem_todos_termos_produto:
            preco = resultado.find_element('class name', 'a8Pemb').text
            preco = preco.replace('R$','').replace(' ', '').replace('.', '').replace(',','.')
            preco = float(preco)

            # se o preço está entre preco_minimo e preco_maximo
            if preco_minimo <= preco <= preco_maximo:
                elemento_referencia = resultado.find_element('class name', 'bONr3b')
                elemento_pai = elemento_referencia.find_element('xpath', '..')
                link = elemento_pai.get_attribute('href')
                lista_ofertas.append((nome, preco, link))
    return lista_ofertas

def busca_buscape(navegador, produto, termos_banidos, preco_minimo, preco_maximo):
    produto = produto.lower()
    lista_termos_nome_produto = produto.split(' ')
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    lista_ofertas = []

    # entrar no google
    navegador.get("https://www.buscape.com.br/")
    
    # entrar na aba shopping
    navegador.find_element('xpath', '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)
     
    # pegar as informações do produto
    while len(navegador.find_elements('class name', 'Select_Select__1S7HV')) < 1:
        time.sleep(1)
    lista_resultados = navegador.find_elements('class name', 'SearchCard_ProductCard_Inner__7JhKb')

    for resultado in lista_resultados:
        nome = resultado.find_element('class name', 'SearchCard_ProductCard_Name__ZaO5o').text
        nome = nome.lower()
        
        # analisar se nome não tem nenhum dos termos banidos
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)

        # analisar se nome tem todos os termos do produto
        tem_todos_termos_produto = verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome)

        # selecionar só os elementos que tem_termos_banidos = False e ao mesmo tempo tem_todos_termos_produto = True
        if not tem_termos_banidos and tem_todos_termos_produto:
            preco = resultado.find_element('class name', 'Text_MobileHeadingS__Zxam2').text
            preco = preco.replace('R$','').replace(' ', '').replace('.', '').replace(',','.')
            preco = float(preco)

            # se o preço está entre preco_minimo e preco_maximo
            if preco_minimo <= preco <= preco_maximo:
                link = resultado.get_attribute('href')
                lista_ofertas.append((nome, preco, link))
    return lista_ofertas

# pesquisar pelo produto 
tabela_ofertas = pd.DataFrame()

for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, 'Nome']
    termos_banidos =  tabela_produtos.loc[linha, 'Termos banidos']
    preco_minimo = tabela_produtos.loc[linha, 'Preço mínimo']
    preco_maximo = tabela_produtos.loc[linha, 'Preço máximo']

    lista_ofertas_google_shopping = busca_google_shopping(navegador, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['Produto', 'Preço', 'Link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_google_shopping])
    else:
        tabela_google_shopping = None
    
    lista_ofertas_buscape = busca_buscape(navegador, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['Produto', 'Preço', 'Link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape])
    else:
        tabela_buscape = None

display(tabela_ofertas)

# exportar para o excel
tabela_ofertas.to_excel('Ofertas.xlsx', index=False)

if len(tabela_ofertas) > 0:
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = "charlespakua@gmail.com"
    mail.Subject = 'Produto(s) Encontrado(s) na faixa de preço desejada'
    mail.HTMLBody = f'''
    <p>Prezados,</p>
    <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada</p>
    {tabela_ofertas.to_html(index=False)}
    <p>Att.,</p>
    <p>Charles.</p>
    '''
    mail.Send()
    navegador.quit()

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.79M/6.79M [00:01<00:00, 4.62MB/s]


,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,3000,4000
1,rtx 3060,zota galax,2500,4500


KeyboardInterrupt: 